In [47]:
import pandas as pd
import numpy as np

In [48]:
import re

In [49]:
import nltk



In [50]:
# Download NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1006)>


False

In [51]:
from sklearn.cluster import KMeans

In [52]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [53]:
def preprocess_text(text):
    # Remove HTML tags (if any)
    text = re.sub(r"<.*?>", "", text)
    
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # print(filtered_tokens)
    # # Lemmatization
    # lemmatizer = WordNetLemmatizer()
    # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # # Join the tokens back into a single string
    # processed_text = " ".join(filtered_tokens)

    
    return filtered_tokens

In [54]:
# amazon=pd.read_csv("Eco_Friendly_Products_Test_Full.xlsx - Sheet1.csv")
amazon=pd.read_csv("Amazon Reviews Validation Data_1.csv")

In [55]:
# a=amazon["Review"]
a=amazon["Reviews"]


# word2vec

In [56]:
processed_series = a.apply(preprocess_text)

In [57]:
from gensim.models import Word2Vec

In [58]:
# Train Word2Vec model
model = Word2Vec(processed_series, vector_size=100, window=5, min_count=1)

# Get word embeddings
word_embeddings = model.wv

In [59]:
# Obtain word vectors
word_vectors = [word_embeddings[word] for word in word_embeddings.index_to_key]

In [60]:
vectors = np.array(word_vectors)

## kmeanns

In [61]:
# Apply K-means clustering
num_clusters = 4  # Number of aspects/topics
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(word_vectors)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=4)

In [62]:
# Get cluster labels
labels = kmeans.labels_

In [63]:
# Assign aspects to words based on cluster labels
word_aspects = {word: label for word, label in zip(word_embeddings.index_to_key, labels)}

In [64]:
# Print word aspects
dict_aspects = {}
for word, aspect in word_aspects.items():
    print(f"Word: {word}")
    print(f"Aspect: {aspect}")
    print()
    dict_aspects[word] = aspect

Word: easy
Aspect: 3

Word: great
Aspect: 2

Word: well
Aspect: 1

Word: use
Aspect: 1

Word: good
Aspect: 1

Word: like
Aspect: 1

Word: quality
Aspect: 3

Word: durable
Aspect: 3

Word: work
Aspect: 1

Word: one
Aspect: 0

Word: really
Aspect: 3

Word: also
Aspect: 1

Word: price
Aspect: 2

Word: time
Aspect: 1

Word: clean
Aspect: 2

Word: nice
Aspect: 1

Word: get
Aspect: 3

Word: product
Aspect: 0

Word: easily
Aspect: 0

Word: sturdy
Aspect: 3

Word: comfortable
Aspect: 3

Word: works
Aspect: 3

Word: water
Aspect: 3

Word: fit
Aspect: 0

Word: wash
Aspect: 2

Word: enough
Aspect: 0

Word: even
Aspect: 3

Word: bags
Aspect: 0

Word: strong
Aspect: 3

Word: would
Aspect: 1

Word: makes
Aspect: 0

Word: light
Aspect: 2

Word: hot
Aspect: 0

Word: hold
Aspect: 0

Word: size
Aspect: 3

Word: keep
Aspect: 0

Word: colors
Aspect: 2

Word: heavy
Aspect: 1

Word: bag
Aspect: 3

Word: keeps
Aspect: 0

Word: color
Aspect: 0

Word: beautiful
Aspect: 2

Word: quickly
Aspect: 0

Word: made
As

In [65]:
from scipy.spatial.distance import cdist

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_

# Create a dictionary to hold the top words for each cluster
top_words_per_cluster = {}

for cluster_num in range(num_clusters):
    # Get the indices of the words that belong to this cluster
    word_indices = np.where(kmeans.labels_ == cluster_num)[0]
    
    # Get the vectors for these words
    word_vectors = np.array([word_embeddings[word_embeddings.index_to_key[index]] for index in word_indices])
    
    # Calculate the distance of these vectors to the cluster center
    distances = cdist(word_vectors, [cluster_centers[cluster_num]], 'euclidean').flatten()
    
    # Get the indices of the sorted distances (ascending), then get the top 5
    closest_word_indices = np.argsort(distances)[:5]
    
    # Map these indices back to words
    top_words = [word_embeddings.index_to_key[word_indices[index]] for index in closest_word_indices]
    
    # Store the top words for this cluster
    top_words_per_cluster[cluster_num] = top_words

# Display the top words for each cluster
for cluster_num, words in top_words_per_cluster.items():
    print(f"Cluster {cluster_num}: {', '.join(words)}")


Cluster 0: interchangable, smells, concentrator, band, rigid
Cluster 1: functional, directly, obstruction, smudge, stray
Cluster 2: corner, steel, unusable, turning, steady
Cluster 3: sooooo, beginning, wise, rained, thread


In [66]:
from sklearn.mixture import GaussianMixture

In [67]:
word_vectors = [word_embeddings[word] for word in word_embeddings.index_to_key]
words = list(word_embeddings.index_to_key)

In [68]:
# Step 1: Fit GMM on the word vectors
gmm = GaussianMixture(n_components=num_clusters, random_state=0).fit(word_vectors)

# Step 2: Compute the probabilities of each word belonging to each component
word_probabilities = gmm.predict_proba(word_vectors)

# Step 3: Identify the most representative words for each cluster
top_words_per_component = {}
for component in range(num_clusters):
    # Find the words with the highest probability of belonging to the current component
    top_indices = np.argsort(word_probabilities[:, component])[::-1][:5]  # Top 5 words
    top_words = [words[i] for i in top_indices]
    top_words_per_component[component] = top_words

# Display the top words for each component
for component, words in top_words_per_component.items():
    print(f"Component {component}: {', '.join(words)}")

Component 0: pop, softness, controllers, seeds, first
Component 1: outlet, bathroom, box, real, cubes
Component 2: proud, de, solution, indoor, still
Component 3: wait, preventing, hand, disintegration, whichever


# glove

In [69]:
def load_glove_model(glove_file):
    """Load the GloVe model from a file."""
    print("Loading GloVe Model")
    with open(glove_file, 'r') as file:
        model = {}
        for line in file:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array([float(val) for val in split_line[1:]])
            model[word] = embedding
        print("Done. {} words loaded!".format(len(model)))
    return model

glove_model = load_glove_model("/Users/divyamsobti/Downloads/glove.6B/glove.6B.100d.txt")


Loading GloVe Model
Done. 400000 words loaded!


In [70]:
def preprocess_text(text):
    # Remove HTML tags (if any)
    text = re.sub(r"<.*?>", "", text)
    
    # Tokenize the text into words
    tokens = nltk.word_tokenize(text)
    
    # Convert to lowercase
    tokens = [token.lower() for token in tokens]
    
    # Remove punctuation
    tokens = [token for token in tokens if token.isalpha()]
    
    # Remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    # print(filtered_tokens)
    # # Lemmatization
    # lemmatizer = WordNetLemmatizer()
    # lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # # Join the tokens back into a single string
    processed_text = " ".join(filtered_tokens)

    
    return processed_text

In [71]:
preprocess_text_g=a.apply(preprocess_text)

In [72]:
def sentence_vector(sentence, model):
    """Generate a sentence vector by averaging the word vectors."""
    words = sentence.split()
    word_vectors = [model[word] for word in words if word in model]
    if len(word_vectors) == 0:
        return np.zeros(100)  # Assuming each word vector has 100 dimensions
    return np.mean(word_vectors, axis=0)

In [73]:
sentences=np.array(preprocess_text_g)

In [74]:
sentence_vectors = np.array([sentence_vector(sentence, glove_model) for sentence in preprocess_text_g])

In [75]:
from sklearn.cluster import KMeans

num_clusters = 4  # Example number of clusters
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(sentence_vectors)
labels = kmeans.labels_

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [76]:
from collections import Counter


In [77]:
cluster_centers = kmeans.cluster_centers_
assigned_clusters = kmeans.labels_

# Calculate the distance of each sentence to its cluster center
distances_to_center = np.sqrt(((sentence_vectors - cluster_centers[assigned_clusters]) ** 2).sum(axis=1))

# Initialize a structure to hold the central sentences of each cluster
central_sentences_indices = {i: [] for i in range(num_clusters)}

for cluster in range(num_clusters):
    # Get indices of sentences in the cluster
    indices = np.where(assigned_clusters == cluster)[0]
    # Sort these indices by their distance to the cluster center, take top 5
    sorted_indices = sorted(indices, key=lambda i: distances_to_center[i])[:5]
    central_sentences_indices[cluster] = sorted_indices

# Extract representative words from these central sentences
cluster_top_words = {}
for cluster, indices in central_sentences_indices.items():
    word_counts = Counter(word for i in indices for word in sentences[i].split())
    # Get the most common 5 words for this cluster
    most_common_words = [word for word, count in word_counts.most_common(5)]
    cluster_top_words[cluster] = most_common_words

# Display the top words for each cluster
for cluster, words in cluster_top_words.items():
    print(f"Cluster {cluster}: {', '.join(words)}")

Cluster 0: like, clock, shower, use, timer
Cluster 1: easy, bit, awkward, carry, shape
Cluster 2: bottom, like, cupholders, wish, bags
Cluster 3: durable


In [78]:
from sklearn.mixture import GaussianMixture

In [79]:
gmm = GaussianMixture(n_components = 4)

In [80]:
gmm.fit(sentence_vectors)

GaussianMixture(n_components=4)

In [81]:
# labels = gmm.predict(sentence_vectors)

In [82]:
# Step 1: Assign each sentence to the most likely GMM component
probs = gmm.predict_proba(sentence_vectors)
highest_prob_clusters = np.argmax(probs, axis=1)

# Step 2: For each cluster, find sentences with the highest probability of belonging to that cluster
cluster_sentences = {}
for cluster in range(gmm.n_components):
    indices = np.where(highest_prob_clusters == cluster)[0]
    # Sort these sentences by their probability of belonging to the cluster
    sorted_indices = sorted(indices, key=lambda i: probs[i, cluster], reverse=True)
    # Take top N sentences to extract words from
    top_sentences = [sentences[i] for i in sorted_indices[:5]]  # N=5 as an example
    cluster_sentences[cluster] = top_sentences

# Step 3: Count words in these top sentences
cluster_top_words = {}
for cluster, top_sentences in cluster_sentences.items():
    word_counts = Counter(word for sentence in top_sentences for word in sentence.split())
    # Get the most common 5 words for this cluster
    most_common_words = [word for word, count in word_counts.most_common(5)]
    cluster_top_words[cluster] = most_common_words

# Step 4: Display the top words for each cluster
for cluster, words in cluster_top_words.items():
    print(f"Cluster {cluster}: {', '.join(words)}")

Cluster 0: durable
Cluster 1: stay, also, diffuser, heating, element
Cluster 2: settings, attractive, perfect, hair, types
Cluster 3: hair, without, frizz, dryer, excellent


# tfidf

In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [84]:
# Create an instance of TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the documents and transform the documents into TF-IDF vectors
tfidf_vectors = vectorizer.fit_transform(preprocess_text_g)

In [85]:
tfidf_vectors

<1181x1711 sparse matrix of type '<class 'numpy.float64'>'
	with 5325 stored elements in Compressed Sparse Row format>

In [86]:
from collections import defaultdict

In [87]:
# Perform clustering using K-means
num_clusters = 4  # Number of clusters
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_vectors)

# Get cluster labels
labels = kmeans.labels_

# Create a dictionary to store documents for each cluster
cluster_documents = defaultdict(list)

for i, doc in enumerate(processed_series):
    cluster_label = labels[i]
    cluster_documents[cluster_label].append(doc)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [88]:
centers = kmeans.cluster_centers_

In [89]:
feature_names = np.array(vectorizer.get_feature_names_out())
top_words_per_cluster_tfidf = {}

for cluster_num in range(num_clusters):
    # Get the centroid for the cluster
    centroid = kmeans.cluster_centers_[cluster_num]
    
    # Get the indices of the top 5 features for this centroid
    top_features_indices = centroid.argsort()[-5:][::-1]
    
    # Get the top words/features for this cluster
    top_words = feature_names[top_features_indices]
    
    # Store the top words for this cluster
    top_words_per_cluster_tfidf[cluster_num] = top_words

# Display the top words for each TF-IDF cluster
for cluster_num, words in top_words_per_cluster_tfidf.items():
    print(f"Cluster {cluster_num}: {', '.join(words)}")

Cluster 0: install, easy, use, seemed, remove
Cluster 1: durable, seems, good, heavy, made
Cluster 2: great, well, good, sturdy, price
Cluster 3: easy, use, clean, wash, super


In [90]:
tfidf_vectors= tfidf_vectors.toarray()

In [91]:
gmm.fit(tfidf_vectors)
labels = gmm.predict(tfidf_vectors)

In [92]:


feature_names = np.array(vectorizer.get_feature_names_out())
top_words_per_component_tfidf = {}

for component_num in range(gmm.n_components):
    # Get the mean vector for the component
    mean_vector = gmm.means_[component_num]
    
    # Get the indices of the top 5 features for this mean vector
    top_features_indices = mean_vector.argsort()[-5:][::-1]
    
    # Get the top words/features for this cluster
    top_words = feature_names[top_features_indices]
    
    # Store the top words for this cluster
    top_words_per_component_tfidf[component_num] = top_words

# Display the top words for each TF-IDF cluster/component
for component_num, words in top_words_per_component_tfidf.items():
    print(f"Component {component_num}: {', '.join(words)}")


Component 0: great, well, comfortable, work, price
Component 1: good, durable, sturdy, quality, price
Component 2: strong, easily, also, tears, thin
Component 3: easy, use, clean, install, wash
